# Imports

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
#%matplotlib notebook
import sys
###
#!pip install openslide-python
#https://github.com/FAU-DLM/python-wsi-preprocessing.git
###
sys.path.append('../preprocessing_pipeline/python-wsi-preprocessing/')
from wsi import slide, filter, tiles, util

import fastai
from fastai.vision import *

import pandas as pd
import numpy as np
import os
from pathlib import Path
from functools import partial, update_wrapper
from tqdm import tqdm_notebook as tqdm
from enum import Enum
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
import cv2
import openslide
from openslide.lowlevel import *
Image.MAX_IMAGE_PIXELS = 10000000000

PATH = Path('/home/Deep_Learner/private/network/datasets/Hypophysenadenome/')
FONT_PATH=PATH/'1984-Happines-Regular.ttf'
WSIS_CORTICOTROP = PATH/'corticotrop'
WSIS_GONADOTROP = PATH/'gonadotrop'
ROIS_CORTICOTROP = PATH/'corticotrop_ROIs'
ROIS_GONADOTROP = PATH/'gonadotrop_ROIs'

TILES_CORTICOTROP_1 = PATH/'tiles_corticotrop_1_scoring_function_1_thresh_0.55'
TILES_CORTICOTROP_2 = PATH/'tiles_corticotrop_2_scoring_function_1_thresh_0.55'
TILES_CORTICOTROP_3 = PATH/'tiles_corticotrop_3_scoring_function_1_thresh_0.4'

TILES_GONADOTROP_1 = PATH/'tiles_gonadotrop_1_scoring_function_1_thresh_0.55'
TILES_GONADOTROP_2 = PATH/'tiles_gonadotrop_2_scoring_function_1_thresh_0.55'
TILES_GONADOTROP_3 = PATH/'tiles_gonadotrop_3_scoring_function_1_thresh_0.4'

ROIS_EXPERIMENTING = PATH/'rois_experimenting'
TILES_EXPERIMENTING = PATH/'tiles_experimenting'

PATH_RELAPSE = Path('/home/Deep_Learner/private/network/datasets/Hypophysenadenome-Rezidive/')
RELAPSE_WSIS_EXPERIMENTING = PATH_RELAPSE/'wsis_experimenting'
RELAPSE_IMAGES_EXPERIMENTING = PATH_RELAPSE/'imgs_experimenting'
RELAPSE_TILES_EXPERIMENTING = PATH_RELAPSE/'tiles_experimenting'

NONE_RELAPSE_WSIS = PATH_RELAPSE/'wsis_non_relapse'
NONE_RELAPSE_IMAGES = PATH_RELAPSE/'imgs_non_relapse'
NONE_RELAPSE_TILES = PATH_RELAPSE/'tiles_non_relapse'

RELAPSE_WSIS = PATH_RELAPSE/'wsis_relapse'
RELAPSE_IMAGES = PATH_RELAPSE/'imgs_relapse'
RELAPSE_TILES = PATH_RELAPSE/'tiles_relapse'


def show(np):
    return util.np_to_pil(np)

Path.ls = lambda x: [p for p in list(x.iterdir()) if '.ipynb_checkpoints' not in p.name]

def show_multiple_images(path, rows = 3, figsize=(128, 64)):
    imgs = [open_image(p) for p in path.ls()]
    show_all(imgs=imgs, r=rows, figsize=figsize)
    
def show_multiple_images_big(path:pathlib.Path):
    for p in path.ls():
        plt.imshow(mpimg.imread(str(p)))
        plt.show()
        
def get_wsi_name_from_path(path):
    split = path.stem.split('-')
    return f'{split[0]}-{split[1]}-{split[2]}-{split[3]}'

def get_roi_name_from_path(path):
        split = path.stem.split('-')
        if split[2] == 'HE':
                return f'{split[0]}-{split[1]}-{split[2]}-{split[3]}-{split[4]}'
        else:
            return f'{split[0]}-{split[1]}-{split[2]}-{split[3]}-{split[4]}-{split[5]}'
        
class AdenomaType(Enum):
    experimenting = 0
    corticotrop = 1
    gonadotrop = 2
    relapse_experimenting = 3
    relapse = 4
    non_relapse = 5

In [2]:
adenomaType = AdenomaType.corticotrop

if adenomaType == AdenomaType.gonadotrop:
    rois_path = ROIS_GONADOTROP
    tiles_path = TILES_GONADOTROP_1
    tiles_path2 = TILES_GONADOTROP_2
    tiles_path3 = TILES_GONADOTROP_3
    
if adenomaType == AdenomaType.corticotrop:
    rois_path = ROIS_CORTICOTROP
    tiles_path = TILES_CORTICOTROP_1
    tiles_path2 = TILES_CORTICOTROP_2
    tiles_path3 = TILES_CORTICOTROP_3
    
if adenomaType == AdenomaType.experimenting:
    wsi_path = ROIS_EXPERIMENTING
    rois_path = ROIS_EXPERIMENTING
    tiles_path = TILES_EXPERIMENTING
    
if adenomaType == AdenomaType.relapse_experimenting:
    wsi_path = RELAPSE_WSIS_EXPERIMENTING
    rois_path = RELAPSE_IMAGES_EXPERIMENTING
    tiles_path = RELAPSE_TILES_EXPERIMENTING
    
if adenomaType == AdenomaType.relapse:
    wsi_path = RELAPSE_WSIS
    rois_path = RELAPSE_IMAGES
    tiles_path = RELAPSE_TILES  
    
if adenomaType == AdenomaType.non_relapse:
    wsi_path = NONE_RELAPSE_WSIS
    rois_path = NONE_RELAPSE_IMAGES
    tiles_path = NONE_RELAPSE_TILES

In [3]:
rois_path.mkdir(exist_ok=True)
tiles_path3.mkdir(exist_ok=True)

# Get tiles from ROIs no tiles have been extracted from yet

In [4]:
rois = list(set([get_roi_name_from_path(p) for p in rois_path.ls()]))

In [5]:
rois[0]

'967-16-III-HE-ROI_7-ACTH'

In [6]:
len(rois)

5373

In [7]:
ts = list(set([get_roi_name_from_path(p) for p in (tiles_path.ls()+tiles_path2.ls())]))

In [8]:
ts[0]

'967-16-III-HE-ROI_7-ACTH'

In [9]:
len(ts)

4176

In [12]:
rois_no_ts_extracted_yet = list(set(rois) - set(ts))

In [13]:
len(rois_no_ts_extracted_yet)

1197

In [14]:
ts = list(set([get_roi_name_from_path(p) for p in (tiles_path.ls()+tiles_path2.ls())]))
paths = []
for p in rois_path.ls():
    b = False
    for t in ts:
        if str(t) in str(p):
            b = True
            continue
    if(not b):
        paths.append(p)

In [15]:
len(paths)

1197

In [17]:
tiles.WsiOrROIToTilesMultithreaded(paths, 
                                   tiles_path3, 
                                   1024,1024, 
                                   is_wsi=False, 
                                   tile_score_thresh=0.4, 
                                   tile_naming_func=tiles.get_roi_name_from_path_pituitary_adenoma_entities)

Starting to process /home/Deep_Learner/private/network/datasets/Hypophysenadenome/corticotrop_ROIs/618-13-II-HE-ROI_35-ACTH.png
Starting to process /home/Deep_Learner/private/network/datasets/Hypophysenadenome/corticotrop_ROIs/786-13-III-HE-ROI_2-ACTH.png
Starting to process /home/Deep_Learner/private/network/datasets/Hypophysenadenome/corticotrop_ROIs/79-14-III-HE-ROI_8-ACTH.png
Starting to process /home/Deep_Learner/private/network/datasets/Hypophysenadenome/corticotrop_ROIs/710-15-III-HE-ROI_20-ACTH.png
Starting to process /home/Deep_Learner/private/network/datasets/Hypophysenadenome/corticotrop_ROIs/1883-12-III-HE-ROI_2-ACTH.png
Starting to process /home/Deep_Learner/private/network/datasets/Hypophysenadenome/corticotrop_ROIs/1427-14-III-HE-ROI_8-ACTH.png
Starting to process /home/Deep_Learner/private/network/datasets/Hypophysenadenome/corticotrop_ROIs/1722-12-Z-HE-ROI_2-ACTH.png
Starting to process /home/Deep_Learner/private/network/datasets/Hypophysenadenome/corticotrop_ROIs/1118

Starting to process /home/Deep_Learner/private/network/datasets/Hypophysenadenome/corticotrop_ROIs/1118-14-III-HE-ROI_22-ACTH.png
/home/Deep_Learner/private/network/datasets/Hypophysenadenome/corticotrop_ROIs/925-15-III-HE-ROI_11-ACTH.png: Number of tiles that will be saved/all possible tiles: 0/12
Starting to process /home/Deep_Learner/private/network/datasets/Hypophysenadenome/corticotrop_ROIs/618-13-II-HE-ROI_138-ACTH.png
/home/Deep_Learner/private/network/datasets/Hypophysenadenome/corticotrop_ROIs/1281-12-Z-HE-ROI_73-ACTH.png: Number of tiles that will be saved/all possible tiles: 0/9
Starting to process /home/Deep_Learner/private/network/datasets/Hypophysenadenome/corticotrop_ROIs/1336-13-III-HE-ROI_25-ACTH.png
/home/Deep_Learner/private/network/datasets/Hypophysenadenome/corticotrop_ROIs/618-13-II-HE-ROI_126-ACTH.png: Number of tiles that will be saved/all possible tiles: 0/6
Starting to process /home/Deep_Learner/private/network/datasets/Hypophysenadenome/corticotrop_ROIs/1281-

/home/Deep_Learner/private/network/datasets/Hypophysenadenome/corticotrop_ROIs/205-12-III-HE-ROI_17-ACTH.png: Number of tiles that will be saved/all possible tiles: 0/4
Starting to process /home/Deep_Learner/private/network/datasets/Hypophysenadenome/corticotrop_ROIs/618-13-II-HE-ROI_14-ACTH.png
/home/Deep_Learner/private/network/datasets/Hypophysenadenome/corticotrop_ROIs/205-12-III-HE-ROI_14-ACTH.png: Number of tiles that will be saved/all possible tiles: 0/20
Starting to process /home/Deep_Learner/private/network/datasets/Hypophysenadenome/corticotrop_ROIs/82-14-II-HE-ROI_5-ACTH.png
/home/Deep_Learner/private/network/datasets/Hypophysenadenome/corticotrop_ROIs/1427-14-III-HE-ROI_29-ACTH.png: Number of tiles that will be saved/all possible tiles: 2/9
Starting to process /home/Deep_Learner/private/network/datasets/Hypophysenadenome/corticotrop_ROIs/2423-14-Z-HE-ROI_21-ACTH.png
/home/Deep_Learner/private/network/datasets/Hypophysenadenome/corticotrop_ROIs/82-14-II-HE-ROI_5-ACTH.png: Nu

/home/Deep_Learner/private/network/datasets/Hypophysenadenome/corticotrop_ROIs/332-17-III-HE-ROI_1-ACTH.png: Number of tiles that will be saved/all possible tiles: 0/4
Starting to process /home/Deep_Learner/private/network/datasets/Hypophysenadenome/corticotrop_ROIs/1773-13-III-HE-ROI_1-ACTH.png
/home/Deep_Learner/private/network/datasets/Hypophysenadenome/corticotrop_ROIs/71-14-III-HE-ROI_53-ACTH.png: Number of tiles that will be saved/all possible tiles: 0/30
Starting to process /home/Deep_Learner/private/network/datasets/Hypophysenadenome/corticotrop_ROIs/618-13-II-HE-ROI_120-ACTH.png
/home/Deep_Learner/private/network/datasets/Hypophysenadenome/corticotrop_ROIs/490-17-III-HE-ROI_10-ACTH.png: Number of tiles that will be saved/all possible tiles: 1/6
Starting to process /home/Deep_Learner/private/network/datasets/Hypophysenadenome/corticotrop_ROIs/618-13-II-HE-ROI_24-ACTH.png
/home/Deep_Learner/private/network/datasets/Hypophysenadenome/corticotrop_ROIs/71-14-III-HE-ROI_12-ACTH.png:

Starting to process /home/Deep_Learner/private/network/datasets/Hypophysenadenome/corticotrop_ROIs/205-12-III-HE-ROI_37-ACTH.png
/home/Deep_Learner/private/network/datasets/Hypophysenadenome/corticotrop_ROIs/71-14-III-HE-ROI_5-ACTH.png: Number of tiles that will be saved/all possible tiles: 0/9
Starting to process /home/Deep_Learner/private/network/datasets/Hypophysenadenome/corticotrop_ROIs/1928-13-III-HE-ROI_1-ACTH.png
/home/Deep_Learner/private/network/datasets/Hypophysenadenome/corticotrop_ROIs/1281-12-Z-HE-ROI_80-ACTH.png: Number of tiles that will be saved/all possible tiles: 0/20
Starting to process /home/Deep_Learner/private/network/datasets/Hypophysenadenome/corticotrop_ROIs/1928-13-IV-HE-ROI_12-ACTH.png
/home/Deep_Learner/private/network/datasets/Hypophysenadenome/corticotrop_ROIs/1281-12-Z-HE-ROI_106-ACTH.png: Number of tiles that will be saved/all possible tiles: 0/6
Starting to process /home/Deep_Learner/private/network/datasets/Hypophysenadenome/corticotrop_ROIs/71-14-III

...

/home/Deep_Learner/private/network/datasets/Hypophysenadenome/corticotrop_ROIs/618-13-II-HE-ROI_63-ACTH.png: Number of tiles that will be saved/all possible tiles: 0/20
Starting to process /home/Deep_Learner/private/network/datasets/Hypophysenadenome/corticotrop_ROIs/1079-13-III-HE-ROI_25-ACTH.png
Starting to process /home/Deep_Learner/private/network/datasets/Hypophysenadenome/corticotrop_ROIs/1928-13-IV-HE-ROI_7-ACTH.png
Starting to process /home/Deep_Learner/private/network/datasets/Hypophysenadenome/corticotrop_ROIs/1079-13-III-HE-ROI_36-ACTH.png
/home/Deep_Learner/private/network/datasets/Hypophysenadenome/corticotrop_ROIs/71-14-III-HE-ROI_69-ACTH.png: Number of tiles that will be saved/all possible tiles: 5/110
/home/Deep_Learner/private/network/datasets/Hypophysenadenome/corticotrop_ROIs/1928-13-IV-HE-ROI_7-ACTH.png: Number of tiles that will be saved/all possible tiles: 0/2
Starting to process /home/Deep_Learner/private/network/datasets/Hypophysenadenome/corticotrop_ROIs/767-15

Starting to process /home/Deep_Learner/private/network/datasets/Hypophysenadenome/corticotrop_ROIs/1928-13-IV-HE-ROI_11-ACTH.png
Starting to process /home/Deep_Learner/private/network/datasets/Hypophysenadenome/corticotrop_ROIs/2423-14-Z-HE-ROI_18-ACTH.png
Starting to process /home/Deep_Learner/private/network/datasets/Hypophysenadenome/corticotrop_ROIs/925-15-III-HE-ROI_12-ACTH.png
/home/Deep_Learner/private/network/datasets/Hypophysenadenome/corticotrop_ROIs/1336-13-II-HE-ROI_15-ACTH.png: Number of tiles that will be saved/all possible tiles: 1/20
/home/Deep_Learner/private/network/datasets/Hypophysenadenome/corticotrop_ROIs/1118-14-III-HE-ROI_8-ACTH.png: Number of tiles that will be saved/all possible tiles: 1/12
/home/Deep_Learner/private/network/datasets/Hypophysenadenome/corticotrop_ROIs/925-15-III-HE-ROI_8-ACTH.png: Number of tiles that will be saved/all possible tiles: 1/4
Starting to process /home/Deep_Learner/private/network/datasets/Hypophysenadenome/corticotrop_ROIs/925-15-

/home/Deep_Learner/private/network/datasets/Hypophysenadenome/corticotrop_ROIs/1079-13-III-HE-ROI_60-ACTH.png: Number of tiles that will be saved/all possible tiles: 0/52
Starting to process /home/Deep_Learner/private/network/datasets/Hypophysenadenome/corticotrop_ROIs/1336-13-II-HE-ROI_12-ACTH.png
/home/Deep_Learner/private/network/datasets/Hypophysenadenome/corticotrop_ROIs/71-14-III-HE-ROI_28-ACTH.png: Number of tiles that will be saved/all possible tiles: 0/15
Starting to process /home/Deep_Learner/private/network/datasets/Hypophysenadenome/corticotrop_ROIs/1079-13-III-HE-ROI_29-ACTH.png
/home/Deep_Learner/private/network/datasets/Hypophysenadenome/corticotrop_ROIs/205-12-III-HE-ROI_86-ACTH.png: Number of tiles that will be saved/all possible tiles: 0/12
Starting to process /home/Deep_Learner/private/network/datasets/Hypophysenadenome/corticotrop_ROIs/1883-13-II-HE-ROI_4-ACTH.png
/home/Deep_Learner/private/network/datasets/Hypophysenadenome/corticotrop_ROIs/1281-12-Z-HE-ROI_62-ACTH

Process ForkPoolWorker-12:
Process ForkPoolWorker-20:
Process ForkPoolWorker-16:
Process ForkPoolWorker-26:
Process ForkPoolWorker-25:
Process ForkPoolWorker-27:
Process ForkPoolWorker-32:
Process ForkPoolWorker-24:
Process ForkPoolWorker-31:


KeyboardInterrupt: 

In [ ]:
import random

In [ ]:
samples = random.choices(tiles_path.ls(), k=100)

In [ ]:
util.show_multiple_images_big(samples)